In [95]:
import sklearn
import pandas as pd
import numpy as np
import boto3
import pprint
import os
import time

from numpy.random import RandomState
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [96]:
bucket = "saurav-ml-sagemaker"
prefix = "ChurnModel/"

In [97]:
data_key = prefix + 'Churn_Modelling.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

data_df = pd.read_csv(data_location)

In [98]:
data_df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [99]:
data_df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [100]:
churn_data = data_df[['CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited']]

In [101]:
churn_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [102]:
churn_data.shape

(10000, 11)

In [103]:
columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [1,2])], remainder='passthrough')
data = np.array(columnTransformer.fit_transform(churn_data), dtype = np.str)

In [104]:
data.shape

(10000, 14)

In [105]:
data

array([['1.0', '0.0', '0.0', ..., '1.0', '101348.88', '1.0'],
       ['0.0', '0.0', '1.0', ..., '1.0', '112542.58', '0.0'],
       ['1.0', '0.0', '0.0', ..., '0.0', '113931.57', '1.0'],
       ...,
       ['1.0', '0.0', '0.0', ..., '1.0', '42085.58', '1.0'],
       ['0.0', '1.0', '0.0', ..., '0.0', '92888.52', '1.0'],
       ['1.0', '0.0', '0.0', ..., '0.0', '38190.78', '0.0']], dtype='<U32')

In [106]:
sc = StandardScaler()
data[:,:-1] = sc.fit_transform(data[:,:-1])

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype <U32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype <U32 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [107]:
data.shape

(10000, 14)

In [108]:
data

array([['0.9972039090469509', '-0.5787359118285802',
        '-0.5738091541336481', ..., '0.970242550937133',
        '0.021886493995389635', '1.0'],
       ['-1.0028039309990795', '-0.5787359118285802',
        '1.7427397119690895', ..., '0.970242550937133',
        '0.21653375188734358', '0.0'],
       ['0.9972039090469509', '-0.5787359118285802',
        '-0.5738091541336481', ..., '-1.030670113400118',
        '0.24068690006349144', '1.0'],
       ...,
       ['0.9972039090469509', '-0.5787359118285802',
        '-0.5738091541336481', ..., '0.970242550937133',
        '-1.0086430751933717', '1.0'],
       ['-1.0028039309990795', '1.7279038324064944',
        '-0.5738091541336481', ..., '-1.030670113400118',
        '-0.1252307125385332', '1.0'],
       ['0.9972039090469509', '-0.5787359118285802',
        '-0.5738091541336481', ..., '-1.030670113400118',
        '-1.0763697553829055', '0.0']], dtype='<U32')

In [117]:
df_final = pd.DataFrame(data)
df_final.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.9972039090469509,-0.5787359118285802,-0.5738091541336481,1.0959875190286517,-1.0959875190286514,-0.3262214220367455,0.29351742289674765,-1.041759679225302,-1.2258476714090163,-0.911583494040172,0.6460916681531842,0.970242550937133,0.021886493995389635,1.0
1,-1.0028039309990795,-0.5787359118285802,1.7427397119690895,1.0959875190286517,-1.0959875190286514,-0.4400359548576646,0.19816383219544578,-1.387537586562431,0.11735002143511637,-0.911583494040172,-1.5477679860172207,0.970242550937133,0.21653375188734358,0.0
2,0.9972039090469509,-0.5787359118285802,-0.5738091541336481,1.0959875190286517,-1.0959875190286514,-1.5367941802228853,0.29351742289674765,1.0329077647974714,1.333053345722891,2.5270566192762067,0.6460916681531842,-1.030670113400118,0.24068690006349144,1.0
3,0.9972039090469509,-0.5787359118285802,-0.5738091541336481,1.0959875190286517,-1.0959875190286514,0.5015206348426664,0.007456650792842043,-1.387537586562431,-1.2258476714090163,0.8077365626180174,-1.5477679860172207,-1.030670113400118,-0.10891791702846762,0.0
4,-1.0028039309990795,-0.5787359118285802,1.7427397119690895,1.0959875190286517,-1.0959875190286514,2.0638837672025563,0.3888710135980495,-1.041759679225302,0.7857278997960621,-0.911583494040172,0.6460916681531842,0.970242550937133,-0.36527578251439885,0.0


In [118]:
df_final[14] = df_final[0]
df_final[0] = df_final[13]


In [124]:
df_final.drop(columns = [13], inplace = True)

In [125]:
df_final.shape

(10000, 14)

In [126]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
0     10000 non-null object
1     10000 non-null object
2     10000 non-null object
3     10000 non-null object
4     10000 non-null object
5     10000 non-null object
6     10000 non-null object
7     10000 non-null object
8     10000 non-null object
9     10000 non-null object
10    10000 non-null object
11    10000 non-null object
12    10000 non-null object
14    10000 non-null object
dtypes: object(14)
memory usage: 1.1+ MB


In [127]:
df_final[0] = (df_final[0]).astype(str).astype(float).astype(int)

In [128]:
df_final.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,14
0,1,-0.5787359118285802,-0.5738091541336481,1.0959875190286517,-1.0959875190286514,-0.3262214220367455,0.29351742289674765,-1.041759679225302,-1.2258476714090163,-0.911583494040172,0.6460916681531842,0.970242550937133,0.021886493995389635,0.9972039090469509
1,0,-0.5787359118285802,1.7427397119690895,1.0959875190286517,-1.0959875190286514,-0.4400359548576646,0.19816383219544578,-1.387537586562431,0.11735002143511637,-0.911583494040172,-1.5477679860172207,0.970242550937133,0.21653375188734358,-1.0028039309990795
2,1,-0.5787359118285802,-0.5738091541336481,1.0959875190286517,-1.0959875190286514,-1.5367941802228853,0.29351742289674765,1.0329077647974714,1.333053345722891,2.5270566192762067,0.6460916681531842,-1.030670113400118,0.24068690006349144,0.9972039090469509
3,0,-0.5787359118285802,-0.5738091541336481,1.0959875190286517,-1.0959875190286514,0.5015206348426664,0.007456650792842043,-1.387537586562431,-1.2258476714090163,0.8077365626180174,-1.5477679860172207,-1.030670113400118,-0.10891791702846762,0.9972039090469509
4,0,-0.5787359118285802,1.7427397119690895,1.0959875190286517,-1.0959875190286514,2.0638837672025563,0.3888710135980495,-1.041759679225302,0.7857278997960621,-0.911583494040172,0.6460916681531842,0.970242550937133,-0.36527578251439885,-1.0028039309990795


In [129]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
0     10000 non-null int64
1     10000 non-null object
2     10000 non-null object
3     10000 non-null object
4     10000 non-null object
5     10000 non-null object
6     10000 non-null object
7     10000 non-null object
8     10000 non-null object
9     10000 non-null object
10    10000 non-null object
11    10000 non-null object
12    10000 non-null object
14    10000 non-null object
dtypes: int64(1), object(13)
memory usage: 1.1+ MB


In [130]:
rng = RandomState()

train = df_final.sample(frac=0.7, random_state=rng)

test = df_final.loc[~df_final.index.isin(train.index)]

In [131]:
rng = RandomState()

validation = test.sample(frac=0.5, random_state=rng)

test = test.loc[~test.index.isin(validation.index)]

In [132]:
train.shape

(7000, 14)

In [133]:
test.shape

(1500, 14)

In [134]:
validation.shape

(1500, 14)

In [135]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7000 entries, 7036 to 9423
Data columns (total 14 columns):
0     7000 non-null int64
1     7000 non-null object
2     7000 non-null object
3     7000 non-null object
4     7000 non-null object
5     7000 non-null object
6     7000 non-null object
7     7000 non-null object
8     7000 non-null object
9     7000 non-null object
10    7000 non-null object
11    7000 non-null object
12    7000 non-null object
14    7000 non-null object
dtypes: int64(1), object(13)
memory usage: 820.3+ KB


In [136]:
train.to_csv('train.csv', index=False, header = False)
test.to_csv('test.csv', index=False, header = False)
validation.to_csv('validation.csv', index=False, header = False)

In [87]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 6 to 9995
Data columns (total 14 columns):
0     1500 non-null object
1     1500 non-null object
2     1500 non-null object
3     1500 non-null object
4     1500 non-null object
5     1500 non-null object
6     1500 non-null object
7     1500 non-null object
8     1500 non-null object
9     1500 non-null object
10    1500 non-null object
11    1500 non-null object
12    1500 non-null object
13    1500 non-null int64
dtypes: int64(1), object(13)
memory usage: 175.8+ KB


In [88]:
validation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 1956 to 9801
Data columns (total 14 columns):
0     1500 non-null object
1     1500 non-null object
2     1500 non-null object
3     1500 non-null object
4     1500 non-null object
5     1500 non-null object
6     1500 non-null object
7     1500 non-null object
8     1500 non-null object
9     1500 non-null object
10    1500 non-null object
11    1500 non-null object
12    1500 non-null object
13    1500 non-null int64
dtypes: int64(1), object(13)
memory usage: 175.8+ KB
